# IMDb

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/IMDB_Logo_2016.svg/640px-IMDB_Logo_2016.svg.png?1623833316814">

- IMDb (an acronym for Internet Movie Database)[1] is an online database of information related to films, television programs, home videos, video games, and streaming content online – including cast, production crew and personal biographies, plot summaries, trivia, ratings, and fan and critical reviews. An additional fan feature, message boards, was abandoned in February 2017. Originally a fan-operated website, the database is now owned and operated by IMDb.com, Inc., a subsidiary of Amazon.

- As of December 2020, IMDb has approximately 7.5 million titles (including episodes) and 10.4 million personalities in its database,[2] as well as 83 million registered users.

- IMDb began as a movie database on the Usenet group "rec.arts.movies" in 1990 and moved to the web in 1993.

- Information ref: https://www.wikiwand.com/en/IMDb

- Now, we're interested in what's going on and finding insight by given data from "https://relational.fit.cvut.cz/dataset/IMDb"

## Proceduces for this project

    1 Business Understanding
    2 Data Preparation
        - Collecting
        - Cleaning
    3 Data Understanding
    4 Describing Data
        - EDA

### Additional Libraries
- pymysql
- pandas_profiling

In [ ]:
! pip install pymysql

In [ ]:
# ! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip 

## Entity Relational Diagram

    A Diagram display the relations between databases is shown here

<img src="https://relational.fit.cvut.cz/assets/img/datasets-generated/imdb_ijs.svg">

<img src="https://media1.tenor.com/images/871791f2c3b6a691428d5eb69001647e/tenor.gif?itemid=16088453">

### Data Preparation section

In [3]:
# Inspects all all tables, are still available, from database

import pymysql.cursors

hostname =  "relational.fit.cvut.cz"
port = 3306
username = "guest"
password = "relational"
db = "imdb_ijs"

connections = pymysql.connect(
    host=hostname,
    port=port,
    user=username,
    password=password,
    db=db,
    charset="utf8mb4",
    cursorclass=pymysql.cursors.DictCursor
)

cursor = connections.cursor()
cursor.execute("show tables;")
tables = cursor.fetchall()
cursor.close()

for ele in tables:
    print(ele)

{'Tables_in_imdb_ijs': 'actors'}
{'Tables_in_imdb_ijs': 'directors'}
{'Tables_in_imdb_ijs': 'directors_genres'}
{'Tables_in_imdb_ijs': 'movies'}
{'Tables_in_imdb_ijs': 'movies_directors'}
{'Tables_in_imdb_ijs': 'movies_genres'}
{'Tables_in_imdb_ijs': 'roles'}


In [4]:
# Extracts tables from database to pandas dataframe

import pandas as pd

def extract_dataframe(table_name: str):
    with connections.cursor() as cursor:
        # select all from given table name without any conditions
        select = "SELECT * FROM " + table_name
        cursor.execute(select)
        result = cursor.fetchall()
        return pd.DataFrame(result)

raw_actors = extract_dataframe("actors")
raw_directors = extract_dataframe("directors")
raw_directorsGen = extract_dataframe("directors_genres")
raw_movies = extract_dataframe("movies")
raw_moviesDirec = extract_dataframe("movies_directors")
raw_moviesGen = extract_dataframe("movies_genres")
raw_roles = extract_dataframe("roles")

# Assigning name
dfs_name = {
    "Actors" : raw_actors,
    "Directors" : raw_directors,
    "Directors genres" : raw_directorsGen,
    "Movies" : raw_movies,
    "Movies-directors" : raw_moviesDirec,
    "Movies-genres" : raw_moviesGen,
    "Roles" : raw_roles 
}

list_raw = [
    raw_actors,
    raw_directors,
    raw_directorsGen,
    raw_movies,
    raw_moviesDirec,
    raw_moviesGen,
    raw_roles
]

In [ ]:
# Exporting raw tables

# raw_actors.to_csv("actors.csv")
# raw_directors.to_csv("directors.csv")
# raw_directorsGen.to_csv("directors_genres.csv")
# raw_movies.to_csv("movies.csv")
# raw_moviesDirec.to_csv("movies_directors.csv")
# raw_moviesGen.to_csv("movies_genres.csv")
# raw_roles.to_csv("roles.csv")

In [5]:
# Print the first few row of each

for name_df, value in dfs_name.items():
    print(name_df)
    print(value.head())

Actors
   id first_name           last_name gender
0   2    Michael  'babeepower' Viera      M
1   3       Eloy         'Chincheta'      M
2   4   Dieguito         'El Cigala'      M
3   5    Antonio    'El de Chipiona'      M
4   6       José        'El Francés'      M
Directors
   id first_name    last_name
0   1       Todd            1
1   2        Les  12 Poissons
2   3    Lejaren     a'Hiller
3   4       Nian            A
4   5   Khairiya    A-Mansour
Directors genres
   director_id        genre  prob
0            2        Short   1.0
1            3        Drama   1.0
2            5  Documentary   1.0
3            6        Drama   1.0
4            6        Short   1.0
Movies
   id                                 name  year  rank
0   0                                  #28  2002   NaN
1   1  #7 Train: An Immigrant Journey, The  2000   NaN
2   2                                    $  1971   6.4
3   3                        $1,000 Reward  1913   NaN
4   4                        $1,000 

## Data Cleaning
    1 Validate each columns are in the right type
    2 Remove missing values
    3 Adjust values error (the value that makes no sense)
    4 Remove duplicates rows
    5 Standardize data (if possible)

In [6]:
# Looking basic information of each tables
# The basic information can tell us the type of each columns, null object

for name_df, df in dfs_name.items():
    print(name_df)
    print(df.info())

Actors
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 817718 entries, 0 to 817717
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          817718 non-null  int64 
 1   first_name  817718 non-null  object
 2   last_name   817718 non-null  object
 3   gender      817718 non-null  object
dtypes: int64(1), object(3)
memory usage: 25.0+ MB
None
Directors
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86880 entries, 0 to 86879
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          86880 non-null  int64 
 1   first_name  86880 non-null  object
 2   last_name   86880 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB
None
Directors genres
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156562 entries, 0 to 156561
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   ---

#### Result
- We inspected that **all columns are in the right type**
- e.g. all id columns are in integer type, rank is in float type, names or genres are in object typr
- There are massively null valus in Movies dataframe

In [8]:
# Is there any data is missing

for name_df, df in dfs_name.items():
    print(name_df)
    print(df.isna().sum())
    

Actors
id            0
first_name    0
last_name     0
gender        0
dtype: int64
Directors
id            0
first_name    0
last_name     0
dtype: int64
Directors genres
director_id    0
genre          0
prob           0
dtype: int64
Movies
id           0
name         0
year         0
rank    321024
dtype: int64
Movies-directors
director_id    0
movie_id       0
dtype: int64
Movies-genres
movie_id    0
genre       0
dtype: int64
Roles
actor_id    0
movie_id    0
role        0
dtype: int64
